Federal University of Minas Gerais

[Bioinformatics and Systems Laboratory](http://bioinfo.dcc.ufmg.br/)

Authors: Lucas Moraes, Prof. Dr. Raquel C. de Melo Minardi

Most recent update date: 07/08/2023


##### 1. **Initial Settings**



---





> 1.1 GPU



In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()




> 1.2 Available memory



In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')





> 1.3 Sync with Google Drive



In [ ]:
from google.colab import drive
drive.mount("/content/drive")




> 1.4 Global variables



In [ ]:
BASE_DIR = "/content/drive/My Drive/"

# Training hyperparameters
BATCH_SIZE = 64
DROPOUT = 0.5
EPOCH = 100
LRN_RATE = 1e-03



> 1.5 Importing libraries



In [ ]:
# Module for interfacing with the operating system
import os

# Module with time functions
import time

# Libraries for machine learning
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Libraries for developing neural networks
import tensorflow as tf
from tensorflow import keras

# Keras modules for developing multi-layer neural networks
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model

from keras.layers import Dense, Activation, Concatenate
from keras.layers import BatchNormalization, Dropout

from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint

from keras.optimizers import Adam

from keras.preprocessing.image import ImageDataGenerator

from keras.utils import load_img, img_to_array

# Specific Keras modules for developing convolutional neural networks
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten

# Libraries for data analysis and matrix operations
import numpy as np
import pandas as pd

# Libraries for data plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

##### 2. **Data preprocessing**



---





> 2.1 Pre-processing of distance maps



In [ ]:
train_datagen = ImageDataGenerator(
                    rescale = 1./255,
                    height_shift_range = 0.1,
                    width_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)



> 2.2 Training Set



In [ ]:
train_data = train_datagen.flow_from_directory(
    BASE_DIR+'Dataset/Variants/Training',
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',
    shuffle=True,
    seed=42
)



> 2.3 Validation Set



In [ ]:
val_data = val_datagen.flow_from_directory(
    BASE_DIR+'Dataset/Variants/Validation',
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',
    shuffle=True,
    seed=42
)


> 2.4 Load the Test Set




In [ ]:
test_data = test_datagen.flow_from_directory(
    DIR_BASE+'Dataset/Variants/Test',
    target_size = (224, 224),
    batch_size = 1,
    class_mode = None,
    shuffle=False,
    seed=42
)

##### 3. **Deep convolutional neural networks-based model**



---





> 3.1 Sequential model



In [ ]:
class VGGNet:
  @staticmethod
  def build(width, height, depth):

    model = Sequential()
    INPUT_SHAPE = (height, width, depth)
    chanDim = -1

  # Convolutional layers

  # 1st layer block
  # CONV => Activation => CONV => Activation => POOL layer set

    model.add(Conv2D(input_shape=INPUT_SHAPE, filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(Conv2D(input_shape=INPUT_SHAPE, filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))

  # 2nd bloco de camadas
  # CONV => Activation => CONV => Activation => POOL layer set

    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))

  # 3rd layer block
  # CONV => Activation => CONV => Activation => POOL layer set

    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))

  # 4th layer block
  # CONV => Activation => CONV => Activation => POOL layer set

    model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))

  # 5th layer block
  # CONV => Activation => CONV => Activation => POOL layer set

    model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(0.25))

  # Vectorization

  # 6th layer blocks
  # Flatten

    model.add(Flatten())

  # Fully connected layers

  # 7th layer block
  # Dense layer

    model.add(Dense(4096))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation('relu'))
    model.add(Dropout(DROPOUT))

    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(DROPOUT))

  # Output Layer
  # Sigmoid classifier

    model.add(Dense(1, activation='sigmoid'))

    return model



> 3.2 Instantiate the model



In [ ]:
model = VGGNet.build(width = 224, height = 224, depth = 3)



> 3.3 Compilation of the model



In [ ]:
OPT = Adam(learning_rate=LRN_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

model.compile(
            optimizer= OPT,
            loss='binary_crossentropy',
            metrics=["accuracy"]
)



> 3.4 Model checkpoint configuration



In [ ]:
checkpoint = ModelCheckpoint(
                filepath = DIR_BASE+'Checkpoint/model-{epoch:02d}-{val_accuracy:.4f}.hdf5',
                monitor = 'val_accuracy',
                mode = 'max',
                save_best_only=True,
                verbose=1
)



> 3.5 Training



In [ ]:
to = time.time()

history = model.fit(train_data, epochs=EPOCH, validation_data=val_data, callbacks = [checkpoint])

t = time.time()

print('Runtime:', str(round(t-to,3)), 's.')

model.save(DIR_BASE+'Models/wuhan_beta-delta_vgg_13_224_224_1e-03_05_100_20230711.h5')



> 3.6 Saving model weights



In [ ]:
model.save_weights(DIR_BASE+"Models/weights_wuhan_beta-delta_vgg_13_224_224_1e-03_05_100_20230711_20230711.h5")



> 3.7 Define a new model from a VGG configuration and load the pre-trained model weights



In [ ]:
model = load_model(DIR_BASE+'Models/wuhan_beta-delta_vgg_13_224_224_1e-03_05_100_20230711.h5')

In [ ]:
config = model.get_config()
model = Sequential.from_config(config)

In [ ]:
model.load_weights(DIR_BASE+"Models/weights_wuhan_beta-delta_vgg_13_224_224_1e-03_05_100_20230711_20230711.h5")

##### 4. **Evaluation of model performance**



---




> 4.1 Import the trained model



In [ ]:
model = load_model(DIR_BASE+'Models/wuhan_beta-delta_vgg_13_224_224_1e-03_05_100_20230711.h5')


> 4.2 Prediction



In [ ]:
test_data.reset()
probabilities = model.predict(test_data,verbose=1)


> 4.3 Classification



In [ ]:
# Definition of true labels for each sample

neutral = np.full(5000,0)
positive = np.full(5000,1)

true = np.concatenate((neutral,positive))

In [ ]:
threshold = 0.5
predicted_class = []
y_pred = []
y_probs = []

for p in probabilidades:
  if p[0]<threshold:
    predicted_class.append('neutral')
    y_pred.append(0)
  else:
    predicted_class.append('positive')
    y_pred.append(1)
  y_probs.append(p[0])

y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

> 4.4 Performance on the test set

In [ ]:
print(classification_report(true, y_pred, labels=[0,1]))

> 4.5 Receiver Operating Curve

> 4.5.1 Confusion matrix

In [ ]:
zeros = np.zeros(2000)
ones = np.ones(2000)
y_test = np.concatenate([zeros, ones])

cm = confusion_matrix(y_test, y_pred)
cm

> 4.5.2 Function definition to obtain the ROC curve

In [ ]:
def calculate_tpr_fpr(y_real, y_pred):
    """
    Calculate True Positive Rate (TPR) and False Positive Rate (FPR) based on confusion matrix.
    """
    cm = confusion_matrix(y_real, y_pred)
    TN, FP, FN, TP = cm.ravel()

    tpr = TP / (TP + FN)
    fpr = FP / (TN + FP)

    return tpr, fpr

def get_n_roc_coordinates(y_real, y_proba, num_points=500):
    """
    Obtain coordinates for the ROC curve.
    """
    tpr_list, fpr_list = [0], [0]

    for i in range(num_points):
        threshold = i / num_points
        y_pred = y_proba > threshold
        tpr, fpr = calculate_tpr_fpr(y_real, y_pred)
        tpr_list.append(tpr)
        fpr_list.append(fpr)

    return tpr_list, fpr_list

def plot_roc_curve(tpr, fpr, scatter=True):
    """
    Plot the ROC curve.
    """
    plt.figure(figsize=(5, 5))

    if scatter:
        sns.scatterplot(x=fpr, y=tpr)
    sns.lineplot(x=fpr, y=tpr)

    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.show()

> 4.5.3 Plotting the ROC curve

In [ ]:
tpr, fpr = get_n_roc_coordinates(y_test, y_probs)
plot_roc_curve(tpr, fpr)

> 4.5.4 Saves predictions in a .csv file


In [ ]:
filenames=test_data.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predito,
                      "Probs":y_probs})
results.to_csv(DIR_BASE+'Results/y505f.csv',index=False)

> 4.6 Calculation of performance metrics

In [ ]:
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

predicted = np.array(predito)

# total number of times the model predicts "positive", with the images belonging to the "positive" class
true_positives = (predicted[0:5000]=="positive").sum()

# total number of times the model predicts "neutral", with the images belonging to the "positive" class
false_negatives = (predicted[0:5000]=="neutral").sum()

# total number of times the model predicts "neutral", with the images belonging to the "neutral" class
true_negatives = (predicted[5001:10000]=="neutral").sum()

# total number of times the model predicts "positive", with the images belonging to the "neutral" class
false_positives = (predicted[5001:10000]=="positive").sum()

# Accuracy
accuracy = (true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
print("Accuracy:", accuracy)

# Error
loss = 1-accuracy
print("Error:",loss)

# Precision
precision = true_positives/(true_positives+false_positives)
print("Precision:",prec)

# Recall
recall = true_positives/(true_positives+false_negatives)
print("Recall:", recall)

# Specificity
specificity = true_negatives/(true_negatives+false_positives)
print("Specificity:",spec)



---

